In [3]:
# eda.ipynb

# -----------------------------
# 1. Imports
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (14,6)


In [4]:
#LOAD ENV PATH
current_dir = os.getcwd()
project_root = current_dir

while not os.path.isfile(os.path.join(project_root, ".env")):
    # Go one directory up
    parent_dir = os.path.abspath(os.path.join(project_root, ".."))
    if parent_dir == project_root:
        raise FileNotFoundError(".env file not found in any parent directory")
    project_root = parent_dir

# Step 2: Load .env from project root
load_dotenv(os.path.join(project_root, ".env"))


True

In [6]:

# -----------------------------
# 2. Load Dataset
# -----------------------------
df = pd.read_parquet(os.getenv("DATA_DIR_PROCESSED") + "/USD_ZAR_2025_07.parquet")

# Inspect first rows
display(df.head())
print(df.info())
print(df.describe())


,exchange,date_time,bid,ask
0,USD/ZAR,20250701 00:00:00.078,17.7147,17.7272
1,USD/ZAR,20250701 00:00:00.141,17.7123,17.7338
2,USD/ZAR,20250701 00:00:00.156,17.7167,17.7267
3,USD/ZAR,20250701 00:00:00.234,17.7185,17.7267
4,USD/ZAR,20250701 00:00:00.234,17.7185,17.7272


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720782 entries, 0 to 6720781
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   exchange   object 
 1   date_time  object 
 2   bid        float64
 3   ask        float64
dtypes: float64(2), object(2)
memory usage: 205.1+ MB
None
                bid           ask
count  6.720782e+06  6.720782e+06
mean   1.775517e+01  1.776158e+01
std    1.463539e-01  1.462219e-01
min    1.744190e+01  1.748770e+01
25%    1.761330e+01  1.761960e+01
50%    1.776970e+01  1.777550e+01
75%    1.786620e+01  1.787200e+01
max    1.823120e+01  1.829920e+01


In [5]:

# -----------------------------
# 3. Data Cleaning
# -----------------------------
# Remove duplicates
df = df.drop_duplicates()

# Convert date_time to datetime
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d %H:%M:%S.%f')

# Sort by timestamp
df = df.sort_values('date_time').reset_index(drop=True)

# Check for missing timestamps
print("Missing timestamps:", df['date_time'].isna().sum())



Missing timestamps: 0


In [6]:
# -----------------------------
# 4. Feature Creation for EDA
# -----------------------------
# Mid-price and spread
df['mid_price'] = (df['bid'] + df['ask']) / 2
df['spread'] = df['ask'] - df['bid']

# Price changes / deltas
df['delta_bid'] = df['bid'].diff()
df['delta_ask'] = df['ask'].diff()
df['delta_mid'] = df['mid_price'].diff()

# Rolling statistics
df['roll_mean_5'] = df['mid_price'].rolling(5).mean()
df['roll_std_50'] = df['mid_price'].rolling(50).std()



In [ ]:
# -----------------------------
# 5. Exploratory Visualizations
# -----------------------------

output_folder = os.getenv("EDA_IMG_PATH")
os.makedirs(output_folder, exist_ok=True)  # create folder if it doesn't exist

# 1. Bid, Ask, Mid price over time
plt.figure()
plt.plot(df['date_time'], df['bid'], label='Bid', alpha=0.7)
plt.plot(df['date_time'], df['ask'], label='Ask', alpha=0.7)
plt.plot(df['date_time'], df['mid_price'], label='Mid Price', alpha=0.7)
plt.legend()
plt.title('USD/ZAR Bid, Ask, Mid Price Over Time')
plt.xlabel('Date Time')
plt.ylabel('Price')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "bid_ask_mid_price.png"))
plt.close()

# 2. Spread over time
plt.figure()
plt.plot(df['date_time'], df['spread'])
plt.title('Spread Over Time')
plt.xlabel('Date Time')
plt.ylabel('Spread (ZAR)')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "spread_over_time.png"))
plt.close()

# 3. Histogram of spread
plt.figure()
sns.histplot(df['spread'], bins=50, kde=True)
plt.title('Spread Distribution')
plt.xlabel('Spread')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "spread_distribution.png"))
plt.close()

# 4. Histogram of mid-price delta
plt.figure()
sns.histplot(df['delta_mid'].dropna(), bins=100, kde=True)
plt.title('Mid-Price Change Distribution')
plt.xlabel('Delta Mid Price')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "delta_mid_distribution.png"))
plt.close()

# 5. Rolling volatility
plt.figure()
plt.plot(df['date_time'], df['roll_std_50'])
plt.title('Rolling 50-Tick Volatility of Mid Price')
plt.xlabel('Date Time')
plt.ylabel('Volatility')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "rolling_volatility.png"))
plt.close()

In [8]:

# -----------------------------
# 6. Insights / Checks
# -----------------------------
print("Dataset duration:", df['date_time'].min(), "to", df['date_time'].max())
print("Total ticks:", len(df))
print("Average spread:", df['spread'].mean())
print("Max spread:", df['spread'].max())
print("Average delta_mid:", df['delta_mid'].mean())
print("Max delta_mid:", df['delta_mid'].max())


Dataset duration: 2025-07-01 00:00:00.078000 to 2025-07-31 23:59:51.306000
Total ticks: 6537132
Average spread: 0.006415747700979567
Max spread: 0.4002999999999979
Average delta_mid: 7.598134410951792e-08
Max delta_mid: 0.04720000000000013
